## Have a Comments - Lista de Documentos:

### Carregar LD e Report do Integra com as informações a serem avaliadas:

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

# pd.set_option('mode.chained_assignment', None)
# Nome dos arquivos para carregar no Pandas

ld_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_5121_D5A_ultima_rev.xlsx'
ld_file_comments = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_5121_D5A_comments.xlsx'
integra_report = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\RE-General Query - Technical Engineering Documents-0000000122.xlsx'

df_LD = pd.read_excel(ld_file, sheet_name="VDRL")
df_LD_comments = pd.read_excel(ld_file_comments, sheet_name="VDRL")
df_integra = pd.read_excel(integra_report, sheet_name="DADOS")


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


#### Pre-processamento dos dados 

In [7]:
df_LD.columns = df_LD.iloc[6]
df_LD = df_LD.iloc[7:]

df_LD_comments.columns = df_LD_comments.iloc[6]
df_LD_comments = df_LD_comments.iloc[7:]

df_LD.rename(columns = {"CLIENT DOCUMENT NUMBER":"CLIENT_DOCUMENT", 
                        "VENDOR DOCUMENT NUMBER":"VENDOR_DOCUMENT", 
                        "DOCUMENT TITLE": "DOCUMENT_TITLE", "PLANNED DATE": "PLANNED_DATE"}, inplace = True)
df_LD_comments.rename(columns = {"Have comment?":"Have_comment", "CLIENT DOCUMENT NUMBER":"CLIENT_DOCUMENT", 
                        "VENDOR DOCUMENT NUMBER":"VENDOR_DOCUMENT", 
                        "DOCUMENT TITLE": "DOCUMENT_TITLE", "PLANNED DATE": "PLANNED_DATE"}, inplace = True)
df_LD = df_LD.dropna(subset=['DOCUMENT_TITLE'])
df_LD_comments = df_LD_comments.dropna(subset=['DOCUMENT_TITLE'])

df_integra.rename(columns = {"CREATION DATE":"CREATION_DATE"}, inplace = True)

colunas_para_manter = ["CLIENT_DOCUMENT", "VENDOR_DOCUMENT", "DOCUMENT_TITLE", "ORIGINATOR ", "PLANNED_DATE"]  # Lista de colunas para  manter
df_LD.drop(columns=[col for col in df_LD.columns if col not in colunas_para_manter], inplace = True)
df_LD.reset_index(drop=True, inplace = True)

colunas_para_manter_VDA = ["Have_comment", "Discipline", "CLIENT_DOCUMENT", "VENDOR_DOCUMENT", "DOCUMENT_TITLE", "ORIGINATOR ", "PLANNED_DATE"]  # Lista de colunas para  manter
df_LD_comments.drop(columns=[col for col in df_LD_comments.columns if col not in colunas_para_manter_VDA], inplace = True)
df_LD_comments.reset_index(drop=True, inplace = True)

# Remover as duas últimas colunas 
df_LD = df_LD.iloc[:, :-2]
df_LD_comments = df_LD_comments.iloc[:, :-2]



### Itera o df_integra e o df_LD_comments inclui novas colunas no dataframe df_LD. Compara o df_LD_comments com o df_LD para buscar informação perdida de Have_a_comment e Discipline.

In [8]:
for row in df_integra.itertuples():
    codigo = row.CODE
    descricao = row.TITLE
    dt_criacao = row.CREATION_DATE
    
    # usando a função .loc para atualizar os valores
    # df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description) & (df_escopo['item_tag'] == item_tag) , "PO_Number"] = pkg_engineer
    df_LD.loc[(df_LD['CLIENT_DOCUMENT'] == codigo), "CODE"] = codigo
    df_LD.loc[(df_LD['CLIENT_DOCUMENT'] == codigo), "TITLE"] = descricao
    df_LD.loc[(df_LD['CLIENT_DOCUMENT'] == codigo), "CREATION_DATE"] = dt_criacao

for row in df_LD_comments.itertuples():
    have_comment = row.Have_comment
    discipline = row.Discipline
    codigo = row.CLIENT_DOCUMENT
    descricao = row.DOCUMENT_TITLE

    # avalia se tem o mesmo código de documento:
    df_LD.loc[(df_LD['CLIENT_DOCUMENT'] == codigo), "CLIENT_DOCUMENT_VDA_cod"] = codigo
    df_LD.loc[(df_LD['CLIENT_DOCUMENT'] == codigo), "DOCUMENT_TITLE_VDA_cod"] = descricao
    df_LD.loc[(df_LD['CLIENT_DOCUMENT'] == codigo), "Have_comment_cod"] = have_comment
    df_LD.loc[(df_LD['CLIENT_DOCUMENT'] == codigo), "Discipline_cod"] = discipline

    df_LD.loc[(df_LD['DOCUMENT_TITLE'] == descricao), "CLIENT_DOCUMENT_VDA_desc"] = codigo
    df_LD.loc[(df_LD['DOCUMENT_TITLE'] == descricao), "DOCUMENT_TITLE_VDA_desc"] = descricao
    df_LD.loc[(df_LD['DOCUMENT_TITLE'] == descricao), "Have_comment_desc"] = have_comment
    df_LD.loc[(df_LD['DOCUMENT_TITLE'] == descricao), "Discipline_desc"] = discipline

# display(df_LD)

### Consolida o Have_a_comment e Discipline numa unica coluna.

In [9]:
# Cria e consolida a coluna 'Have_a_comment' a partir de uma das duas colunas "Have_a_comment_cod" ou "Have_a_comment_desc"
df_LD['Have_comment'] = np.where(df_LD['Have_comment_cod'].notnull(),
                                 df_LD['Have_comment_cod'],
                                 df_LD['Have_comment_desc'])

# Cria e consolida a coluna 'Discipline' a partir de uma das duas colunas "Discipline_cod" ou "Discipline_desc"
df_LD['Discipline'] = np.where(df_LD['Discipline_cod'].notnull(),
                                 df_LD['Discipline_cod'],
                                 df_LD['Discipline_desc'])



### Salva DataFrame num arquivo excel

In [11]:
avanco_ld_cesar = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_5121_D5A_have_a_comment.xlsx'

with pd.ExcelWriter(avanco_ld_cesar) as writer:
    df_LD.to_excel(writer, sheet_name='LD', index=False)
print(f'Arquivo {avanco_ld_cesar} salvo em: ', Path.cwd())

Arquivo C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\LD_5121_D5A_have_a_comment.xlsx salvo em:  c:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs
